In [1]:
import numpy as np
import pandas as pd

In [2]:
candidates = pd.read_csv('candidates_ge15.csv')

In [25]:
winners = candidates[candidates['result_desc'] == 'won']

In [26]:
winners['party'].replace('BARISAN NASIONAL (BN)', 'Barisan Nasional (BN)', inplace=True)
winners['party'].replace('PAKATAN HARAPAN (PH)', 'Pakatan Harapan (PH)', inplace=True)
winners['party'].replace('PARTI TINDAKAN DEMOKRATIK (DAP)', 'Democratic Action Party (DAP)', inplace=True)
winners['party'].replace('IKATAN DEMOKRATIK MALAYSIA (MUDA)', 'Malaysian United Democratic Alliance (MUDA)', inplace=True)
winners['party'].replace('PERIKATAN NASIONAL (PN)', 'Perikatan Nasional (PN)', inplace=True)
winners['party'].replace('PARTI ISLAM SE MALAYSIA (PAS)', 'Parti Islam Se-Malaysia (PAS)', inplace=True)
winners['party'].replace('GABUNGAN PARTI SARAWAK (GPS)', 'Gabungan Parti Sarawak (GPS)', inplace=True)
winners['party'].replace('PARTI GABUNGAN RAKYAT SABAH (GRS)', 'Gabungan Rakyat Sabah (GRS)', inplace=True)
winners['party'].replace('PARTI WARISAN SABAH (WARISAN)', 'Parti Warisan (WARISAN)', inplace=True)
winners['party'].replace('PARTI KESEJAHTERAAN DEMOKRATIK MASYARAKAT (KDM)', 'Parti Kesejahteraan Demokratik Masyarakat (KDM)', inplace=True)
winners['party'].replace('PARTI BANGSA MALAYSIA (PBM)', 'Parti Bangsa Malaysia (PBM)', inplace=True)
winners['party'].replace('BEBAS (BEBAS)', 'Independent (BEBAS)', inplace=True)

/tmp/ipykernel_29279/2576665368.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winners['party'].replace('BARISAN NASIONAL (BN)', 'Barisan Nasional (BN)', inplace=True)
/tmp/ipykernel_29279/2576665368.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winners['party'].replace('PAKATAN HARAPAN (PH)', 'Pakatan Harapan (PH)', inplace=True)
/tmp/ipykernel_29279/2576665368.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winners['pa

In [13]:
results[['parlimen', 'majoriti']]

NameError: name 'results' is not defined

In [5]:
census = pd.read_csv('census_parlimen.csv')

In [15]:
max(census['population_total'])

687609

In [28]:
results = pd.read_csv('results_parlimen_ge15.csv')
winners_complete = pd.merge(winners, results[['parlimen', 'undi_keluar_peti', 'majoriti', 'pengundi_jumlah', 'peratus_keluar']], on='parlimen', how='left')
winners_complete = pd.merge(winners_complete, census[['parlimen', 'population_total', 'ethnicity_proportion_bumi', 'ethnicity_proportion_chinese', 'ethnicity_proportion_indian', 'ethnicity_proportion_other']], on='parlimen', how='left')

In [29]:
winners_complete.fillna(np.nan, inplace=True)

In [30]:
def convert_to_int(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return value
winners_complete['majoriti'] = winners_complete['majoriti'].apply(convert_to_int)

In [31]:
winners_complete['undi_keluar_peti'] = winners_complete['undi_keluar_peti'].apply(convert_to_int)

In [40]:
winners_complete[winners_complete['parlimen'] == 'P.017 Padang Serai']

,state,parlimen,ballot_order,name,name_display,age,sex,ethnicity,party,votes,...,new_mp,undi_keluar_peti,majoriti,pengundi_jumlah,peratus_keluar,population_total,ethnicity_proportion_bumi,ethnicity_proportion_chinese,ethnicity_proportion_indian,ethnicity_proportion_other
16,Kedah,P.017 Padang Serai,4,Azman Bin Nasrudin,DATO' CIKGU AZMAN,50.0,male,bumiputera,Perikatan Nasional (PN),51637,...,1,NaN,NaN,133867,NaN,261937,70.114592,14.160656,15.204471,0.52028


In [38]:
winners_complete.to_csv('wrangled_winners_ge15.csv', index=False)

In [2]:
voters = pd.read_csv('voters_ge15.csv')

In [14]:
ages = voters[voters.columns[4:22]].melt().groupby('variable', as_index=False)['value'].sum()

In [31]:
voters_age = pd.DataFrame()
for index, row in ages.iterrows():
    unpacked = row['variable'].split('_')
    if len(unpacked) == 2:
        entry = {'gender': [unpacked[0]], 'age_group': [unpacked[1]], 'population': [row['value']]}
    else:
        entry = {'gender': [unpacked[0]], 'age_group': [f'{unpacked[1]}-{unpacked[2]}'], 'population': [row['value']]}
    entry = pd.DataFrame(entry)
    voters_age = pd.concat([voters_age, entry], ignore_index=True)

In [32]:
voters_age

,gender,age_group,population
0,female,18-20,676070
1,female,21-29,2248215
2,female,30-39,2297015
3,female,40-49,1816016
4,female,50-59,1537394
5,female,60-69,1187031
6,female,70-79,596341
7,female,80-89,214438
8,female,90+,49631
9,male,18-20,717479


In [33]:
voters_age.to_csv('wrangled_voters_age_ge15.csv', index=False)

In [90]:
ge15 = pd.read_csv('candidates_ge15.csv')
ge14 = pd.read_csv('candidates_ge14.csv')

In [91]:
ge15 = ge15[ge15['result'] == 1]
ge14 = ge14[ge14['result'] == 1]

In [8]:
ge15['party'].unique()

array(['PERIKATAN NASIONAL (PN)', 'PAKATAN HARAPAN (PH)',
       'PARTI ISLAM SE MALAYSIA (PAS)', 'BARISAN NASIONAL (BN)',
       'IKATAN DEMOKRATIK MALAYSIA (MUDA)', 'BEBAS (BEBAS)',
       'PARTI KESEJAHTERAAN DEMOKRATIK MASYARAKAT (KDM)',
       'PARTI WARISAN SABAH (WARISAN)',
       'PARTI GABUNGAN RAKYAT SABAH (GRS)',
       'PARTI TINDAKAN DEMOKRATIK (DAP)', 'GABUNGAN PARTI SARAWAK (GPS)',
       'PARTI BANGSA MALAYSIA (PBM)'], dtype=object)

In [9]:
ge14['party'].unique()

array(['BARISAN NASIONAL (BN)', 'PARTI KEADILAN RAKYAT (PKR)',
       'PARTI ISLAM SE MALAYSIA (PAS)', 'BEBAS (BEBAS)',
       'PARTI WARISAN SABAH (WARISAN)', 'PARTI TINDAKAN DEMOKRATIK (DAP)',
       'PARTI SOLIDARITI TANAH AIRKU (SOLIDARITI)'], dtype=object)

In [92]:
parties = {
    'PERIKATAN NASIONAL (PN)': 'Perikatan Nasional (PN)',
    'PAKATAN HARAPAN (PH)': 'Pakatan Harapan (PH)',
    'PARTI ISLAM SE MALAYSIA (PAS)': 'Perikatan Nasional (PN)',
    'BARISAN NASIONAL (BN)': 'Barisan Nasional (BN)',
    'IKATAN DEMOKRATIK MALAYSIA (MUDA)': 'Pakatan Harapan (PH)',
#     'BEBAS (BEBAS)': 'Independent (BEBAS)',
   #'PARTI KESEJAHTERAAN DEMOKRATIK MASYARAKAT (KDM)': 'Gabungan Rakyat Sabah (GRS)',
   #'PARTI WARISAN SABAH (WARISAN)': 'Parti Warisan (WARISAN)',
   #'PARTI GABUNGAN RAKYAT SABAH (GRS)': 'Gabungan Rakyat Sabah (GRS)',
   'PARTI TINDAKAN DEMOKRATIK (DAP)': 'Pakatan Harapan (PH)',
   #'GABUNGAN PARTI SARAWAK (GPS)': 'Gabungan Parti Sarawak (GPS)',
   #'PARTI BANGSA MALAYSIA (PBM)': 'Parti Bangsa Malaysia (PBM)',
    'PARTI KEADILAN RAKYAT (PKR)': 'Pakatan Harapan (PH)',
    #'PARTI SOLIDARITI TANAH AIRKU (SOLIDARITI)': 'Gabungan Rakyat Sabah (GRS)'
}

In [93]:
for index, row in ge14.iterrows():
    if row['party'] in parties:
        ge14.at[index, 'party'] = parties[row['party']]
        
for index, row in ge15.iterrows():
    if row['party'] in parties:
        ge15.at[index, 'party'] = parties[row['party']]

In [94]:
ge14 = ge14[ge14['party'].isin(['Barisan Nasional (BN)', 'Pakatan Harapan (PH)', 'Perikatan Nasional (PN)'])]
ge15 = ge15[ge15['party'].isin(['Barisan Nasional (BN)', 'Pakatan Harapan (PH)', 'Perikatan Nasional (PN)'])]

In [95]:
ge14['state'].replace('W.P. Kuala Lumpur', 'Wilayah Persekutuan', inplace=True)
ge15['state'].replace('W.P. Kuala Lumpur', 'Wilayah Persekutuan', inplace=True)

ge14['state'].replace('W.P. Labuan', 'Wilayah Persekutuan', inplace=True)
ge15['state'].replace('W.P. Labuan', 'Wilayah Persekutuan', inplace=True)

ge14['state'].replace('W.P. Putrajaya', 'Wilayah Persekutuan', inplace=True)
ge15['state'].replace('W.P. Putrajaya', 'Wilayah Persekutuan', inplace=True)

In [96]:
ge14_gb = ge14.groupby(['state', 'party'])['party'].count()

In [97]:
# Create a MultiIndex with all possible combinations of states and parties
all_states = ge14_gb.index.get_level_values('state').unique()
all_parties = ge14_gb.index.get_level_values('party').unique()
all_combinations = pd.MultiIndex.from_product([all_states, all_parties], names=['state', 'party'])

# Reindex the grouped DataFrame with the MultiIndex, filling missing values with zeroes
ge14_fin = ge14_gb.reindex(all_combinations, fill_value=0).reset_index(name='count')

In [98]:
ge15_gb = ge15.groupby(['state', 'party'])['party'].count()

In [99]:
# Create a MultiIndex with all possible combinations of states and parties
all_states = ge15_gb.index.get_level_values('state').unique()
all_parties = ge15_gb.index.get_level_values('party').unique()
all_combinations = pd.MultiIndex.from_product([all_states, all_parties], names=['state', 'party'])

# Reindex the grouped DataFrame with the MultiIndex, filling missing values with zeroes
ge15_fin = ge15_gb.reindex(all_combinations, fill_value=0).reset_index(name='count')

In [104]:
ge14_15 = pd.merge(ge14_fin, ge15_fin, on=['state', 'party'])
ge14_15.columns = ['state', 'party', '14', '15']

In [105]:
ge14_15 = pd.melt(ge14_15, id_vars=['state', 'party'], var_name='election', value_name='seats')
col_order = ['seats', 'state', 'election', 'party']
ge14_15 = ge14_15[col_order]

In [113]:
ge14_15['election'] = ge14_15['election'].astype(int)

In [114]:
ge14_15.to_csv('wrangled_ge14_15.csv', index=False)

In [115]:
ge14_15['election']

0     14
1     14
2     14
3     14
4     14
      ..
79    15
80    15
81    15
82    15
83    15
Name: election, Length: 84, dtype: int64

In [7]:
census.columns

Index(['state', 'parlimen', 'code_state', 'code_parlimen', 'year', 'area_km2',
       'population_total', 'nationality_citizen', 'nationality_non_citizen',
       'sex_male', 'sex_female', 'ethnicity_proportion_bumi',
       'ethnicity_proportion_chinese', 'ethnicity_proportion_indian',
       'ethnicity_proportion_other', 'age_proportion_0_14',
       'age_proportion_15_64', 'age_proportion_65_above',
       'age_proportion_18_above', 'housing_total', 'household_total',
       'household_size_avg', 'live_births', 'live_births_male',
       'live_births_female', 'deaths', 'deaths_male', 'deaths_female',
       'labour_participation_rate', 'labour_unemployment_rate',
       'income_median', 'income_avg', 'expenditure_avg', 'gini',
       'poverty_incidence', 'sme_small', 'sme_micro', 'sme_medium',
       'businesses_agriculture', 'businesses_crops', 'businesses_livestock',
       'businesses_fisheries', 'businesses_forestry', 'businesses_mining',
       'businesses_manufacturing', 'busi

In [12]:
social = census[['parlimen', 'live_births', 'deaths', 'labour_unemployment_rate', 'income_median', 'poverty_incidence', 'gini']].melt('parlimen')
utilities = census[['parlimen', 'housing_total', 'utilities_pipedwater_home', 'utilities_pipedwater_public', 'utilities_pipedwater_other', 'utilities_electricity_home', 'utilities_electricity_none']].melt('parlimen')
businesses = census[['parlimen', 'businesses_agriculture', 'businesses_forestry', 'businesses_mining', 'businesses_manufacturing', 'businesses_construction', 'businesses_services']].melt('parlimen')

In [14]:
social['variable'].unique()

array(['live_births', 'deaths', 'labour_unemployment_rate',
       'income_median', 'poverty_incidence', 'gini'], dtype=object)